In [1]:
import numpy as np
import pandas as pd
import pymongo

In [3]:
class MongoServer():
    credentials = None
    conn = None
    db = None
    collections = {}
    def __init__(self, run = False, credentials = "credentials.txt", db_name = "agile_data_science_group_3"):
        if run:
            if not self.connect2Mongo(credentials):
                print("Connection to server Failed.")
            
            if not self.connect2DataBase(db_name):
                print("Connection Data Base Failed.")
    """Rotine to connect to Mongo DB"""    
    def connect2Mongo(self, credentials = "credentials.txt"):
        try:
            #use your database name, user and password here:
            with open(credentials, 'r', encoding='utf-8') as f:
                [name,password,url,dbname]=f.read().splitlines()
            conn=pymongo.MongoClient("mongodb://{}:{}@{}/{}".format(name,password,url,dbname))
            self.conn = conn
            return True
        except pymongo.errors.ConnectionFailure as e:
            return False
    
    """Routine to connect to a Data Base"""
    def connect2DataBase(self, db_name = "agile_data_science_group_3"):
        try:
            self.db = self.conn[db_name]
            return True
        except:
            return False
    
    """Return the available collections in a data base"""
    def listOfCollections (self):
        return self.db.collection_names()
    
    """Routine to get one collection of the Data Base"""
    # return the collection
    def getCollectionFromServer (self, name_collection):
        if name_collection in self.db.collection_names():
            self.collections[name_collection] = self.db.get_collection(name_collection)
            print ("Collection ",name_collection," Update in Local.")
            return True
        return False
    
    """ Take the collection from the local copy"""
    def getCollection(self, name_collection):
        if not name_collection in self.collections:
            if not self.getCollectionFromServer(name_collection):
                return False
        return self.collections[name_collection]
    
    """ Gets all the items of the collection"""
    def getItems(self, name_collection):
        if not name_collection in self.collections:
            if not self.getCollectionFromServer(name_collection):
                return False
        return [element for element in self.collections[name_collection].find()]
    
    """Query in one Collection"""
    def searchInCollection(self, name_collection, field, patro):
        return [element for element in self.collections[name_collection].find({field:patro})]
    
    """Query in all the Collections"""
    def searchInDB (self, field, patro):
        query = {}
        for collection in self.db.collection_names():
            print(collection)
            query[collection] = [element for element in self.db.get_collection(collection).find({field:patro})]
        return query
        
MS = MongoServer(True)        
"""
recipes = MS.getCollection('recipes')
print(recipes.find_one())

ratings = MS.getCollection('ratings')
print(ratings.find_one())

ratings = MS.getCollection('ingredients')
print(ratings.find_one())
"""
users = MS.getCollection('users')
print(users.find_one())


print(MS.searchInDB('_id', users.find_one()['_id']))


Collection  users  Update in Local.
{'_id': ObjectId('5a1aa2cb2cfaa80d6ff251f3'), 'user_name': 'Chauncey Steffen', 'email': 'Chauncey.Steffen@email.com', 'password': 'Chauncey'}
test
ingredients
system.indexes
users_recipes_ratings
users
recipes
recipes_ingredients
ratings
{'test': [], 'ingredients': [], 'system.indexes': [], 'users_recipes_ratings': [], 'users': [{'_id': ObjectId('5a1aa2cb2cfaa80d6ff251f3'), 'user_name': 'Chauncey Steffen', 'email': 'Chauncey.Steffen@email.com', 'password': 'Chauncey'}], 'recipes': [], 'recipes_ingredients': [], 'ratings': []}


In [4]:
len(MS.getItems('ratings'))

Collection  ratings  Update in Local.


118

In [ ]:
class Recommender:
    
    def __init__(self):
        server = MongoServer(True)
        
    def searchRecepieWithIngredients(self):
        pass
    
    def searchUsersById(self):
        pass
    
    def computeRecommenderMatrix(self):
        pass
    
    def computeRecomendation(self):
        pass
    
    def bestRated(self, n):
        pass
    
    def collaborativeFiltering(self, n):
        pass
        
